## Import Libraries

In [1121]:
import pandas as pd
import numpy as np
from sklearn.utils import resample

## Import Tables

In [1122]:
# Doug's Path
diagnoses_icd = pd.read_csv('data/mimic-iv-0.4/hosp/diagnoses_icd.csv.gz', compression='gzip')
d_icd_diagnoses = pd.read_csv('data/mimic-iv-0.4/hosp/d_icd_diagnoses.csv.gz', compression='gzip')
admissions = pd.read_csv('data/mimic-iv-0.4/core/admissions.csv.gz', compression='gzip')
d_labitems = pd.read_csv('data/mimic-iv-0.4/hosp/d_labitems.csv.gz', compression='gzip')

# # Stephen's Path
# path = "D:\\Bootcamp\\MIMIC IV\\"
# admissions = pd.read_csv(path + "core\\admissions.csv.gz", compression='gzip')
# diagnoses_icd = pd.read_csv(path + "hosp\\diagnoses_icd.csv.gz", compression='gzip')
# d_icd_diagnoses = pd.read_csv(path + "hosp\\d_icd_diagnoses.csv.gz", compression='gzip')
# d_labitems = pd.read_csv(path + "hosp\\d_labitems.csv.gz", compression='gzip')

## Pull codes related to Acute Kidney Failure

In [1123]:
# Congestive Heart Failure
# chf_codes = list(map(str, d_icd_diagnoses[(d_icd_diagnoses['long_title'].str.lower().str.contains('congestive')) & (d_icd_diagnoses['long_title'].str.lower().str.contains('heart'))]['icd_code']))

kidney_list = list(d_icd_diagnoses[(d_icd_diagnoses['long_title'].str.lower().str.contains('kidney')) & (d_icd_diagnoses['long_title'].str.lower().str.contains('failure'))]['icd_code'])
pd.DataFrame(diagnoses_icd[diagnoses_icd['icd_code'].isin(kidney_list)].icd_code.value_counts()).reset_index().head(20).merge(d_icd_diagnoses, left_index = True, left_on = "index", right_on = 'icd_code', how = 'left')

# ICD Codes we care about
# 5849, N179, 5845, N170
akf_codes = ['5849   ', 'N179   ', '5845   ', 'N170   ']


## Start filtering down to only records where AKF is True

In [1124]:
diagnoses_icd_akf = diagnoses_icd[diagnoses_icd['icd_code'].isin(akf_codes)]


In [1125]:
diagnoses_icd_akf.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version
76,19586042,23279761,2,5849,9
517,16108683,21607477,4,5845,9
537,14411859,24976204,12,5845,9
826,17752053,24437021,2,5849,9
893,16569548,24870770,5,5849,9


## Generate new Admissions table with hospital stay duration and AKF flag

In [1126]:
admissions_akf = admissions[['subject_id', 'hadm_id', 'admittime', 'dischtime', 'ethnicity']]

admissions_akf['AKF_subj'] = np.where(admissions_akf.subject_id.isin(diagnoses_icd_akf.subject_id), 1, 0)
admissions_akf['AKF_adm'] = np.where(admissions_akf.hadm_id.isin(diagnoses_icd_akf.hadm_id), 1, 0)
admissions_akf['time_spent'] = pd.to_datetime(admissions_akf['dischtime']) - pd.to_datetime(admissions_akf['admittime'])


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [1127]:
admissions_akf.head(10)

,subject_id,hadm_id,admittime,dischtime,ethnicity,AKF_subj,AKF_adm,time_spent
0,12427812,21593330,2184-01-06 11:51:00,2184-01-10 11:45:00,UNKNOWN,0,0,3 days 23:54:00
1,14029832,22059088,2120-01-18 01:28:00,2120-01-20 16:13:00,OTHER,0,0,2 days 14:45:00
2,14495017,22484010,2175-01-28 15:41:00,2175-01-29 16:00:00,WHITE,0,0,1 days 00:19:00
3,13676048,23865469,2193-01-19 05:27:00,2193-01-24 18:59:00,WHITE,0,0,5 days 13:32:00
4,13831972,27763544,2131-01-27 04:03:00,2131-01-27 05:39:00,WHITE,0,0,0 days 01:36:00
5,18523038,25414328,2142-08-26 17:14:00,2142-08-27 10:00:00,WHITE,0,0,0 days 16:46:00
6,16705931,20580522,2174-10-24 11:30:00,2174-10-24 18:45:00,WHITE,1,0,0 days 07:15:00
7,19963742,24951506,2171-07-04 15:58:00,2171-07-05 13:37:00,UNKNOWN,0,0,0 days 21:39:00
8,10903424,22568585,2181-01-31 13:09:00,2181-01-31 13:42:00,WHITE,0,0,0 days 00:33:00
9,15030422,21975876,2149-09-21 23:54:00,2149-09-22 03:59:00,WHITE,0,0,0 days 04:05:00


## Sample Target Patients for Modeling

In [1128]:
target_sample = admissions_akf[['subject_id', 'AKF_subj']].drop_duplicates()

target_sample = resample(target_sample, n_samples = 10000, replace = False, stratify = target_sample.AKF_subj, random_state = 0)

print('Number of unique Patients: ', target_sample.subject_id.nunique())
print('\nSample Breakdown:')
print(target_sample.AKF_subj.value_counts())

Number of unique Patients:  10000

Sample Breakdown:
0    8846
1    1154
Name: AKF_subj, dtype: int64


In [1129]:
# identify first akf visit for NON-AKF patients
sample_non = target_sample[target_sample.AKF_subj==0]
admissions_non = admissions[admissions.subject_id.isin(sample_non.subject_id)]
admissions_non['admittime'] = pd.to_datetime(admissions_non.admittime)
first_non = admissions_non.loc[admissions_non.groupby('subject_id').admittime.idxmin()]
# sanity checks
print(first_non.shape)                 # number of rows
print(first_non.subject_id.nunique())  # matches number of patients
print(first_non.hadm_id.nunique())     # matches number of visits

# pull out hospital visits for target non-akf samples
sample_non = sample_non.merge(first_non, how='inner', on='subject_id')
sample_non = sample_non[['subject_id', 'hadm_id', 'AKF_subj']]
sample_non = sample_non.rename({'AKF_subj':'AKF'}, axis=1)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


(8846, 15)
8846
8846


In [1130]:
# identify first akf visit for all AKF patients
sample_akf = target_sample[target_sample.AKF_subj==1]
first_akf = admissions_akf[admissions_akf.subject_id.isin(sample_akf.subject_id)]
first_akf['admittime'] = pd.to_datetime(first_akf.admittime)
first_akf = first_akf.loc[first_akf[first_akf.AKF_adm==1].groupby('subject_id').admittime.idxmin()]
first_akf = first_akf[['subject_id', 'hadm_id', 'AKF_adm']]
# sanity checks
print(np.mean(first_akf.AKF_adm))   # make sure everyone has AKF
print(first_akf.shape)                 # make sure number of rows...
print(first_akf.subject_id.nunique())  # ...matches number of patients
print(first_akf.hadm_id.nunique())     # ...matches number of visits

# pull out hospital visits for target akf samples
sample_akf = sample_akf.merge(first_akf, how='inner', on='subject_id')
sample_akf = sample_akf[['subject_id', 'hadm_id', 'AKF_adm']]
sample_akf = sample_akf.rename({'AKF_adm':'AKF'}, axis=1)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


1.0
(1154, 3)
1154
1154


In [1131]:
# merge back together and write to file
admissions_sample = pd.concat([sample_akf, sample_non])
admissions_sample = admissions_sample.reset_index(drop=True)
print(admissions_sample.shape)                  # sanity check
print(admissions_sample.subject_id.nunique())   # sanity check

(10000, 3)
10000


Generate pickle file so we all work on the exact same data

In [1132]:
admissions_sample.to_pickle('admissions_sample.pkl')

In [13]:
# admissions_sample = pd.read_pickle("admissions_sample.pkl")

## Pull Chart and Lab data only where it matches our sample patient list

## We have Decided not to use Chart data after exploring it further

In [14]:
# %%time #run this for next time

# chunksize = 3*(10 ** 6)
# counter=0
# chartevents_sample = []
# for chunk in pd.read_csv('data/mimic-iv-0.4/icu/chartevents.csv.gz', compression='gzip', chunksize=chunksize):
#     chartevents_sample.append(chunk[chunk['subject_id'].isin(list(admissions_sample.subject_id))])
#     counter+=1
#     print(f'chunk {counter} processed')

# chartevents_final = pd.concat(chartevents_sample)


In [15]:
# chartevents_final.to_pickle("chartevents_final.pkl")

In [16]:
# chartevents_final = pd.read_pickle("chartevents_final.pkl")

In [17]:
# chartevents_final.shape

(7602370, 10)

Load Lab Events table

In [1133]:
%%time

chunksize = 3*(10 ** 6)
counter=0
lab_events_sample = []
for chunk in pd.read_csv('data/mimic-iv-0.4/hosp/labevents.csv.gz', compression='gzip', chunksize=chunksize):
    lab_events_sample.append(chunk[chunk['subject_id'].isin(list(admissions_sample.subject_id))])
    counter+=1
    print(f'chunk {counter} processed')

lab_events_final = pd.concat(lab_events_sample)
# 3min 52s


chunk 1 processed
chunk 2 processed
chunk 3 processed


<string>:2: DtypeWarning: Columns (7,9,12) have mixed types.Specify dtype option on import or set low_memory=False.


chunk 4 processed
chunk 5 processed
chunk 6 processed
chunk 7 processed
chunk 8 processed
chunk 9 processed


<string>:2: DtypeWarning: Columns (9,12) have mixed types.Specify dtype option on import or set low_memory=False.


chunk 10 processed


...

chunk 11 processed
chunk 12 processed
chunk 13 processed
chunk 14 processed
chunk 15 processed


<string>:2: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.


chunk 16 processed
chunk 17 processed
chunk 18 processed
chunk 19 processed
chunk 20 processed
chunk 21 processed
chunk 22 processed
chunk 23 processed


<string>:2: DtypeWarning: Columns (7,9) have mixed types.Specify dtype option on import or set low_memory=False.


chunk 24 processed
chunk 25 processed
chunk 26 processed
chunk 27 processed
chunk 28 processed
chunk 29 processed
chunk 30 processed
chunk 31 processed
chunk 32 processed
chunk 33 processed
chunk 34 processed
chunk 35 processed
chunk 36 processed
chunk 37 processed
chunk 38 processed
chunk 39 processed
chunk 40 processed
chunk 41 processed
CPU times: user 4min 25s, sys: 24.3 s, total: 4min 49s
Wall time: 4min 52s


In [1134]:
# lab_events_final.to_pickle("lab_events_final.pkl")

In [20]:
lab_events_final = pd.read_pickle("lab_events_final.pkl")

In [1135]:
lab_events_final.shape

(4570117, 15)

## Manipulating the Data

Take only records related to our sample patients

Filter lab events based on first hospital visit of our sample population

In [1136]:
admissions_sample = pd.read_pickle("admissions_sample.pkl")

In [1137]:
# lab_events_sampled = lab_events_final.copy()
%time lab_events_sampled = lab_events_final[lab_events_final['hadm_id'].isin(admissions_sample.hadm_id)]


CPU times: user 1.32 s, sys: 913 ms, total: 2.24 s
Wall time: 2.75 s


Check how many patients have had each test done

In [1138]:
lab_events_sampled.groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(10)

itemid
51221    6119
51265    6002
51222    5956
51301    5949
51248    5948
51249    5948
51250    5948
51279    5948
51277    5948
50971    5397
Name: subject_id, dtype: int64

Most common tests performed on patients with AKF

In [1139]:
lab_events_sampled[lab_events_sampled['subject_id'].isin(list(admissions_sample[admissions_sample['AKF'] == 1].subject_id))].groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(10)


itemid
50912    1140
50971    1139
50902    1138
50983    1138
51006    1138
50882    1137
50868    1137
50931    1136
51221    1135
51265    1134
Name: subject_id, dtype: int64

Item Black List

In [1257]:
item_black_list = [50920]
# 50920 - test results are text in comments - need to come back and figure out how to handle this


Take only the X most commonly performed tests

In [1258]:
top_test_num = 40

lab_events_valid = lab_events_sampled[~lab_events_sampled['itemid'].isin(item_black_list)]

# On patients with AKF
itemid_sub_sample = lab_events_valid[lab_events_valid['subject_id'].isin(list(admissions_sample[admissions_sample['AKF'] == 1].subject_id))].groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(top_test_num).reset_index().rename(columns = {'index' : 'itemid'}).itemid.to_list()
# On all sample patients
# itemid_sub_sample = lab_events_sampled.groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(top_test_num).reset_index().rename(columns = {'index' : 'itemid'}).itemid.to_list()

lab_events_sampled_sub = lab_events_sampled[lab_events_sampled['itemid'].isin(itemid_sub_sample)]
# lab_events_sampled_sub = lab_events_sampled[lab_events_sampled['itemid'].isin([51466])]

lab_events_sampled_sub


,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
1695,30338499,12455866,24494044.0,13622007,51221,2135-02-16 04:07:00,2135-02-16 04:50:00,34.7,34.7,%,34.0,45.0,NaN,ROUTINE,NaN
1696,30338500,12455866,24494044.0,13622007,51222,2135-02-16 04:07:00,2135-02-16 04:50:00,11.1,11.1,g/dL,11.2,15.7,abnormal,ROUTINE,NaN
1697,30338501,12455866,24494044.0,13622007,51248,2135-02-16 04:07:00,2135-02-16 04:50:00,31.4,31.4,pg,26.0,32.0,NaN,ROUTINE,NaN
1698,30338502,12455866,24494044.0,13622007,51249,2135-02-16 04:07:00,2135-02-16 04:50:00,32.0,32.0,g/dL,32.0,37.0,NaN,ROUTINE,NaN
1699,30338503,12455866,24494044.0,13622007,51250,2135-02-16 04:07:00,2135-02-16 04:50:00,98,98.0,fL,82.0,98.0,NaN,ROUTINE,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122202390,29500021,12390274,26496227.0,62980950,51275,2183-10-23 06:25:00,2183-10-23 08:14:00,94.6,94.6,sec,22.0,35.0,abnormal,ROUTINE,NaN
122202391,29500022,12390274,26496227.0,80463807,51237,2183-10-24 06:50:00,2183-10-24 08:44:00,2.5,2.5,NaN,0.9,1.1,abnormal,ROUTINE,NaN
122202392,29500023,12390274,26496227.0,80463807,51274,2183-10-24 06:50:00,2183-10-24 08:44:00,26.0,26.0,sec,10.4,13.4,abnormal,ROUTINE,NaN
122202393,29500024,12390274,26496227.0,80463807,51275,2183-10-24 06:50:00,2183-10-24 08:44:00,95.4,95.4,sec,22.0,35.0,abnormal,ROUTINE,NaN


In [1259]:
lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == 50920].comments.value_counts()

Series([], Name: comments, dtype: int64)

## Code to look into specific test values

In [1260]:
# lab_events_sampled_sub.groupby(['subject_id', 'hadm_id', 'itemid']).count()


In [1261]:
lab_events_sampled_sub_tiny = lab_events_sampled_sub[(lab_events_sampled_sub['subject_id'] == 19988493) & (lab_events_sampled_sub['hadm_id'] == 25600709.0) & (lab_events_sampled_sub['itemid'].isin([51237]))]#.valuenum.describe()
# lab_events_sampled_sub.loc[(lab_events_sampled_sub['subject_id'] == 19988493) & (lab_events_sampled_sub['hadm_id'] == 25600709.0) & (lab_events_sampled_sub['itemid'].isin([51478, 51484])),]



In [1262]:
# lab_events_sampled_sub_tiny['comments'] = lab_events_sampled_sub_tiny['comments'].replace(np.nan, 'NA')

lab_events_sampled_sub_tiny


,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
116765206,124205250,19988493,25600709.0,29865343,51237,2116-09-26 23:36:00,2116-09-27 00:26:00,1.1,1.1,NaN,0.9,1.1,NaN,STAT,NaN
116765231,124205276,19988493,25600709.0,1870784,51237,2116-09-27 13:35:00,2116-09-27 14:09:00,1.2,1.2,NaN,0.9,1.1,abnormal,STAT,NaN
116765292,124205338,19988493,25600709.0,57401788,51237,2116-09-28 03:53:00,2116-09-28 05:02:00,1.0,1.0,NaN,0.9,1.1,NaN,ROUTINE,NaN
116765394,124205451,19988493,25600709.0,75955093,51237,2116-09-30 06:20:00,2116-09-30 08:16:00,1.0,1.0,NaN,0.9,1.1,NaN,ROUTINE,NaN


## Special updates for Urine sample tests

In [1263]:
print(lab_events_sampled_sub[(lab_events_sampled_sub['itemid'] == 51484)].value.unique())

print(lab_events_sampled_sub[(lab_events_sampled_sub['itemid'] == 51478)].value.unique())


['10' '40' '150' '80' '15' '50' nan 'TR' 'NEG' '20' '100' ' ']
['100' '300' '250' '70' '1000' '150' nan '200' 'NEG' 'TR' '>1000' '500'
 ' ']


In [1264]:
# lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>21.8', '>19.2', 'ERROR', 'UNABLE TO REPORT'])), ]


In [1265]:
# Other invalid value imputation
# For now set them all to normal/most common value
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin([np.nan, 'ERROR', 'UNABLE TO REPORT'])), ['value', 'valuenum']] = 1.1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>21.8'])), ['value', 'valuenum']] = 22
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>19.2'])), ['value', 'valuenum']] = 20

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['>150'])), ['value', 'valuenum']] = 175

# Should we just delete these?
# For 5k patients there are 6 records here
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51274) &  (lab_events_sampled_sub['value'].isin(['ERROR'])), ['value', 'valuenum']] = 11

# Should we just delete these?
# For 5k patients there are 1 records here
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['UNABLE TO REPORT'])), ['value', 'valuenum']] = 31

# Clumsy..
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['34..3'])), ['value', 'valuenum']] = 34.3


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [1292]:
# Used to investigate bad data

lab_events_sampled_sub[lab_events_sampled_sub['value'] == 'Hazy']


,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
29206683,837862,10071770,20058526.0,43821983,51506,2167-12-06 22:35:00,2167-12-07 00:38:00,Hazy,NaN,NaN,NaN,NaN,NaN,STAT,NaN
29209799,3778019,10308297,27825244.0,21148621,51506,2189-01-23 15:32:00,2189-01-23 15:50:00,Hazy,NaN,NaN,NaN,NaN,NaN,STAT,NaN
29209840,3812668,10312004,28618375.0,27772278,51506,2111-11-08 00:44:00,2111-11-08 01:14:00,Hazy,NaN,NaN,NaN,NaN,NaN,STAT,NaN
29211496,5514860,10455613,27463431.0,69865966,51506,2192-04-12 15:42:00,2192-04-12 16:36:00,Hazy,NaN,NaN,NaN,NaN,NaN,STAT,NaN
29211497,5514877,10455613,27463431.0,31718314,51506,2192-04-12 16:05:00,2192-04-12 16:35:00,Hazy,NaN,NaN,NaN,NaN,NaN,STAT,NaN
29212511,6565399,10543835,29816199.0,12733735,51506,2193-01-19 08:28:00,2193-01-19 09:17:00,Hazy,NaN,NaN,NaN,NaN,NaN,STAT,NaN
29216171,10099394,10831490,25326327.0,33086536,51506,2127-01-20 02:18:00,2127-01-20 02:29:00,Hazy,NaN,NaN,NaN,NaN,NaN,STAT,NaN
29219268,13163543,11071368,26751790.0,7552103,51506,2137-11-14 20:36:00,2137-11-14 21:18:00,Hazy,NaN,NaN,NaN,NaN,NaN,STAT,NaN
29222726,16549016,11343910,26218486.0,65301522,51506,2190-09-07 22:25:00,2190-09-07 22:40:00,Hazy,NaN,NaN,NaN,NaN,NaN,STAT,NaN
29223489,17226732,11398464,27871803.0,19429645,51506,2116-02-20 23:38:00,2116-02-21 00:51:00,Hazy,NaN,NaN,NaN,NaN,NaN,STAT,NaN


In [1315]:
# Used to investigate bad data
pd.set_option('display.max_rows', 200)

# lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == 51237].value.value_counts().reset_index().sort_values(by = 'index')
# lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == 51514][['value', 'comments']].drop_duplicates()
lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == 51506][['flag', 'comments', 'value', 'valuenum', 'ref_range_lower', 'ref_range_upper']].drop_duplicates()


# 51466 - measures blood in urine; value and comments sometimes differ; need to add mapping
# 51486 - same as above, leukocytes in the urine
# 51487 - same as above, nitrite in the urine
# 51464 - same as above, bilirubin in the urine
# 51484 - same as above, ketone in the urine
# 51478 - same as above, glucose in the urine
# 51514 - same as above, urobilinogen in the urine
# 51492 - same as above, protein in the urine
# 51506 - same as above, urine appearance

# lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
# lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0


,flag,comments,value,valuenum,ref_range_lower,ref_range_upper
83770,NaN,Clear.,0,0.0,NaN,NaN
1149471,NaN,Hazy.,1,1.0,NaN,NaN
9637718,NaN,CLEAR.,0,0.0,NaN,NaN
9639299,abnormal,Cloudy*.,2,2.0,NaN,NaN
9639950,NaN,Cloudy.,2,2.0,NaN,NaN
9740128,abnormal,Hazy*.,1,1.0,NaN,NaN
29206683,NaN,NaN,1,1.0,NaN,NaN
29206684,NaN,NaN,2,2.0,NaN,NaN
29206703,NaN,NaN,0,0.0,NaN,NaN
29210338,abnormal,NaN,2,2.0,NaN,NaN


In [1316]:
%%time

# 51466
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['TR', 'TR.  ', 'TR*.  '])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['SM', 'SM .  ', 'SM*.  '])), ['value', 'valuenum']] = 2
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['MOD', 'MOD.  ', 'MOD*.  '])), ['value', 'valuenum']] = 3
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['LG', 'LG.  ', 'LG*.  ', 'LGE'])), ['value', 'valuenum']] = 4

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51514
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', 'NORMAL.  ', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin(['>8'])), ['value', 'valuenum']] = 10
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin(['>12.  '])), ['value', 'valuenum']] = 15

# 51464
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['SM', 'SM .  ', 'SM*.  '])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['MOD', 'MOD.  ', 'MOD*.  '])), ['value', 'valuenum']] = 2
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['LG', 'LG.  ', 'LG*.  ', 'LGE'])), ['value', 'valuenum']] = 3

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51487
# Should Pos be marked abnormal?
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['value'].isin(['POS.  ', 'POS', 'POS*.  '])), ['value', 'valuenum']] = 1

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51492
# Should Pos be marked abnormal?
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['TR', 'TR.  ', 'TR*.  '])), ['value', 'valuenum']] = 10
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].str.contains('TR.')), ['value', 'valuenum']] = 10
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['>300'])), ['value', 'valuenum']] = 350
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['>600.  ', '>600*.  '])), ['value', 'valuenum']] = 700

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51486
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['TR', 'TR.  ', 'TR*.  '])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['SM', 'SM .  ', 'SM*.  '])), ['value', 'valuenum']] = 2
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['MOD', 'MOD.  ', 'MOD*.  '])), ['value', 'valuenum']] = 3
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['LG', 'LG.  ', 'LG*.  ', 'LGE'])), ['value', 'valuenum']] = 4

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51506
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['Clear.  ', 'CLEAR.  ', 'Clear', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['SlHazy'])), ['value', 'valuenum']] = 0.5
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['Hazy.  ', 'Hazy*.  ', 'Hazy', 'HAZY', 'HAZY*.  '])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['Cloudy*.  ', 'Cloudy.  ', 'Cloudy', 'CLO', 'CLOUDY'])), ['value', 'valuenum']] = 2

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


CPU times: user 3.97 s, sys: 108 ms, total: 4.08 s
Wall time: 4.11 s


In [1269]:
# lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466)]['value'].map({'' : 0.0, 'NEG' : 0.0, 'TR' : 1.0, 'SM' : 2.0, 'MOD' : 3.0, 'LG' : 4.0, 'LGE' : 4.0}).fillna(0.0)
# lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466)]['value']
# = lab_events_sampled_sub['GarageFinish'].map({'NA' : 0.0, 'Unf' : 1.0, 'RFn' : 2.0, 'Fin' : 3.0})


In [1317]:
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].isin([np.nan, ' ', 'TR', 'NEG'])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].isin([np.nan, ' ', 'TR', 'NEG'])), ['value', 'valuenum']] = 0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# What are good replacement values for these?
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].isin(['>80'])), ['value', 'valuenum']] = 120
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].isin(['>1000'])), ['value', 'valuenum']] = 1250

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['<1.005'])), ['value', 'valuenum']] = 1.000
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['<=1.005'])), ['value', 'valuenum']] = 1.000
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.030'])), ['value', 'valuenum']] = 1.035
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.035'])), ['value', 'valuenum']] = 1.040
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.050'])), ['value', 'valuenum']] = 1.055
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>=1.035'])), ['value', 'valuenum']] = 1.040
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin([' '])), ['value', 'valuenum']] = 1.015

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>13.4'])), ['value', 'valuenum']] = 15

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51491) & (lab_events_sampled_sub['value'].isin([' '])), ['value', 'valuenum']] = 6.0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.070'])), ['value', 'valuenum']] = 1.080

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'


In [1318]:
# Check on differences between value and valuenum columns

lab_events_sampled_sub.loc[((lab_events_sampled_sub['value'].astype(str) + str('.0')) != lab_events_sampled_sub['valuenum'].astype(str)) & ((lab_events_sampled_sub['value'].astype(str) + str('0')) != lab_events_sampled_sub['valuenum'].astype(str)) & ((lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str)) & ((lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str) + str('0')) & ((str('0') + lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str)), ]


,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
336856,33996144,12745561,20731073.0,57802208,51498,2181-01-08 20:52:00,2181-01-08 20:59:00,1.007,1.007,,1.001,1.035,NaN,STAT,NaN
381840,34543514,12785796,24422068.0,27729972,51498,2169-10-08 21:50:00,2169-10-08 22:32:00,1.007,1.007,,1.001,1.035,NaN,STAT,NaN
459851,35481972,12868507,28608077.0,5597239,51498,2149-02-22 13:08:00,2149-02-22 14:10:00,1.017,1.017,,1.001,1.035,NaN,ROUTINE,NaN
517552,35979020,12907424,25601761.0,54048977,51498,2117-10-22 11:08:00,2117-10-22 12:41:00,1.009,1.009,,1.001,1.035,NaN,STAT,NaN
565211,36414065,12945367,26553474.0,47170199,51498,2166-07-20 03:28:00,2166-07-20 04:29:00,1.019,1.019,,1.001,1.035,NaN,ROUTINE,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120951761,16272155,11319328,29695432.0,17340353,51498,2142-06-18 22:06:00,2142-06-18 22:43:00,1.017,1.017,,1.001,1.035,NaN,STAT,NaN
121164400,18401469,11500075,29528054.0,53881996,51498,2130-02-06 23:57:00,2130-02-07 04:10:00,1.027,1.027,,1.001,1.035,NaN,ROUTINE,NaN
121405813,20996771,11706286,26677566.0,76625408,51498,2200-12-20 11:41:00,2200-12-20 12:02:00,1.017,1.017,,1.001,1.035,NaN,ROUTINE,NaN
121517405,22034085,11791735,26645051.0,65960542,51498,2181-02-22 10:03:00,2181-02-22 10:46:00,1.019,1.019,,1.001,1.035,NaN,ROUTINE,NaN


In [1200]:
# lab_events_sampled_sub.loc[(lab_events_sampled_sub['subject_id'] == 19988493) & (lab_events_sampled_sub['hadm_id'] == 25600709.0) & (lab_events_sampled_sub['itemid'].isin([51478, 51484])),]


## Get Aggregate Test Values

In [1319]:
%time lab_events_sampled_sub_grouped = lab_events_sampled_sub.groupby(['subject_id', 'hadm_id', 'itemid']).agg({'itemid' : ['count'], 'valuenum' : ['min', 'max', 'mean'], 'flag' : ['count'], 'ref_range_lower' : ['min'], 'ref_range_upper' : ['min']})
# 'comments' : ['unique'], 


CPU times: user 299 ms, sys: 41.4 ms, total: 341 ms
Wall time: 341 ms


In [1320]:
lab_events_sampled_sub_grouped.reset_index(inplace = True)
lab_events_sampled_sub_grouped.columns = ['subject_id', 'hadm_id', 'itemid', 'count', 'min', 'max', 'mean', 'abn_percent', 'range_min', 'range_max']
# 'comments', 


In [1321]:
# lab_events_sampled_sub_grouped[lab_events_sampled_sub_grouped['itemid'] == 51221]

missing_vals = lab_events_sampled_sub_grouped.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()

min     84
max     84
mean    84
dtype: int64

In [1322]:
lab_events_sampled_sub_grouped['below_min'] = np.where(lab_events_sampled_sub_grouped['min'] < lab_events_sampled_sub_grouped['range_min'], lab_events_sampled_sub_grouped['range_min'] - lab_events_sampled_sub_grouped['min'], 0)
lab_events_sampled_sub_grouped['above_max'] = np.where(lab_events_sampled_sub_grouped['max'] > lab_events_sampled_sub_grouped['range_max'], lab_events_sampled_sub_grouped['max'] - lab_events_sampled_sub_grouped['range_max'], 0)
lab_events_sampled_sub_grouped['abn_percent'] = lab_events_sampled_sub_grouped['abn_percent'] / lab_events_sampled_sub_grouped['count']


In [1323]:
lab_events_sampled_sub_grouped

,subject_id,hadm_id,itemid,count,min,max,mean,abn_percent,range_min,range_max,below_min,above_max
0,10001877,21320596.0,50868,2,11.00,13.00,12.000,0.0,8.0,20.0,0.00,0.0
1,10001877,21320596.0,50882,2,28.00,32.00,30.000,0.0,22.0,32.0,0.00,0.0
2,10001877,21320596.0,50902,2,103.00,106.00,104.500,0.0,96.0,108.0,0.00,0.0
3,10001877,21320596.0,50912,2,1.30,1.40,1.350,1.0,0.5,1.2,0.00,0.2
4,10001877,21320596.0,50931,1,116.00,116.00,116.000,1.0,70.0,100.0,0.00,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...
163150,19999565,20486927.0,51250,2,90.00,91.00,90.500,0.0,82.0,98.0,0.00,0.0
163151,19999565,20486927.0,51265,2,320.00,354.00,337.000,0.0,150.0,440.0,0.00,0.0
163152,19999565,20486927.0,51277,2,12.90,13.30,13.100,0.0,10.5,15.5,0.00,0.0
163153,19999565,20486927.0,51279,2,3.33,3.98,3.655,1.0,4.2,5.4,0.87,0.0


## Merge Admissions data so we can group by ethnicity/gender to grab average range min and max

In [46]:
# Here is where we merge Admissions

# lab_events_sampled_sub_grouped.merge(d_icd_diagnoses, left_index = True, left_on = "index", right_on = 'icd_code', how = 'left')


In [1324]:
# For now aggregate all patients together

lab_range_min_max = lab_events_sampled_sub_grouped.groupby('itemid').agg({'range_min' : 'mean', 'range_max' : 'mean'})
lab_range_min_max.reset_index(inplace = True)

lab_range_dic = dict(zip(lab_range_min_max['itemid'], zip(lab_range_min_max['range_min'], lab_range_min_max['range_max'])))
lab_range_dic


{50861: (0.0, 40.0),
 50863: (37.513502285002076, 126.29663481512256),
 50868: (8.372040586245772, 19.560315670800453),
 50878: (0.0, 40.0),
 50882: (22.0, 32.0),
 50885: (0.0, 4.160468031759298),
 50893: (8.40543572044939, 10.289322691976224),
 50902: (96.0, 108.0),
 50912: (0.44546661608499916, 1.1454666160849576),
 50931: (69.89815902859381, 100.82941637289463),
 50960: (1.5984241908008092, 2.5936967632029453),
 50970: (2.7192450360027816, 4.530045821514294),
 50971: (3.3293866963124787, 5.144080044468837),
 50983: (133.36922503725782, 145.36922503725782),
 51006: (6.0, 20.0),
 51146: (0.0, 1.588577291381669),
 51221: (37.32194803072397, 49.36509233534891),
 51222: (12.54833781061071, 16.729415715245025),
 51237: (0.9000000000000562, 1.0999999999999375),
 51244: (20.01812585499316, 48.47127222982216),
 51248: (26.296065904505717, 31.90652320107599),
 51249: (31.32901815736382, 35.65803631472764),
 51250: (81.62609280430397, 98.37390719569603),
 51265: (150.0, 425.1449516827724),
 51

Pivot the table so we have feature columns related to test results

In [1207]:
lab_events_sampled_sub_grouped[lab_events_sampled_sub_grouped['hadm_id'] == 23248240.0]

,subject_id,hadm_id,itemid,count,min,max,mean,abn_percent,range_min,range_max,below_min,above_max


In [1325]:
lab_events_sampled_sub_grouped['new_index'] = list(zip(lab_events_sampled_sub_grouped['subject_id'], lab_events_sampled_sub_grouped['hadm_id']))


In [1326]:
%time lab_events_sampled_pivot = lab_events_sampled_sub_grouped.pivot(index = 'new_index', columns = 'itemid', values = ['min', 'max', 'mean', 'abn_percent', 'below_min', 'above_max'])
# 'comments', 


CPU times: user 78.7 ms, sys: 15.3 ms, total: 93.9 ms
Wall time: 96 ms


In [1327]:
pd.options.display.max_columns = 500

lab_events_sampled_pivot = lab_events_sampled_pivot.reset_index()
lab_events_sampled_pivot['subject_id'], lab_events_sampled_pivot['hadm_id'] = zip(*lab_events_sampled_pivot['new_index'])
lab_events_sampled_pivot.drop(['new_index'], axis = 1, inplace = True)


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [52]:
# missing_vals = lab_events_sampled_pivot.isnull().sum(axis = 0) 
# missing_vals[missing_vals > 0].sort_values()

## Start Data Imputation

Copy table, retain original for Random Forest

In [1328]:
lab_events_impute = lab_events_sampled_pivot.copy()

In [54]:
# lab_events_sampled_pivot = lab_events_impute

In [55]:
# d_labitems = pd.read_csv('data/mimic-iv-0.4/hosp/d_labitems.csv.gz', compression='gzip')


In [1329]:
# lab_events_impute.columns

d_labitems[d_labitems['itemid'].isin(itemid_sub_sample)]

,itemid,label,fluid,category,loinc_code
52,50861,Alanine Aminotransferase (ALT),Blood,Chemistry,1742-6
76,50863,Alkaline Phosphatase,Blood,Chemistry,6768-6
115,50868,Anion Gap,Blood,Chemistry,1863-0
153,50878,Asparate Aminotransferase (AST),Blood,Chemistry,1920-8
190,51146,Basophils,Blood,Hematology,704-7
212,50882,Bicarbonate,Blood,Chemistry,1963-8
222,51464,Bilirubin,Urine,Hematology,5770-3
229,50885,"Bilirubin, Total",Blood,Chemistry,1975-2
258,51466,Blood,Urine,Hematology,NaN
282,50893,"Calcium, Total",Blood,Chemistry,2000-8


In [1332]:
%%time

np.random.seed(0)

for labitem in lab_range_dic:
    for ind in lab_events_sampled_pivot[lab_events_sampled_pivot[('mean', labitem)].isnull()].index:
        val_max = lab_range_dic[labitem][1]
        val_min = lab_range_dic[labitem][0]

        val_ave = (val_max + val_min) / 2
        val_std = (val_max - val_ave) * .333

        ran_vals = np.random.normal(val_ave, val_std, 50)
        impute_min = min(ran_vals)
        impute_max = max(ran_vals)
        impute_mean = np.mean(ran_vals)
        
        val_min = lab_events_sampled_pivot[('min', labitem)][ind]
        val_max = lab_events_sampled_pivot[('max', labitem)][ind]
        val_mean = lab_events_sampled_pivot[('mean', labitem)][ind]
        
        lab_events_sampled_pivot[('min', labitem)][ind] = np.where(np.isnan(val_min), impute_min, val_min)
        lab_events_sampled_pivot[('max', labitem)][ind] = np.where(np.isnan(val_max), impute_max, val_max)
        lab_events_sampled_pivot[('mean', labitem)][ind] = np.where(np.isnan(val_mean), impute_mean, val_mean)
#         patient[('mean', labitem)].replace(np.nan, impute_mean)
#         patient[('abn_count', labitem)].replace(np.nan, 0)
# 5k patients
# 10 items - 2.5s
# 20 items - 6s
# 30 items - 28.6s

# 10k patients
# 40 items - 1m 34s

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 1min 33s, sys: 621 ms, total: 1min 33s
Wall time: 1min 34s


In [1333]:
for lab_itemid in lab_range_dic:
    lab_events_sampled_pivot[( 'above_max', lab_itemid)] = lab_events_sampled_pivot[( 'above_max', lab_itemid)].replace(np.nan, 0)
    lab_events_sampled_pivot[( 'below_min', lab_itemid)] = lab_events_sampled_pivot[( 'below_min', lab_itemid)].replace(np.nan, 0)
    lab_events_sampled_pivot[( 'abn_percent', lab_itemid)] = lab_events_sampled_pivot[( 'abn_percent', lab_itemid)].replace(np.nan, 0)
#     lab_events_sampled_pivot[( 'comments', lab_itemid)] = lab_events_sampled_pivot[( 'comments', lab_itemid)].replace(np.nan, np.array(np.nan))


In [1334]:
missing_vals = lab_events_sampled_pivot.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()

Series([], dtype: int64)

In [1335]:
lab_events_sampled_pivot

min                                                         \
itemid      50861       50863      50868      50878      50882     50885   
0        2.997088   51.875587  11.000000   7.192662  28.000000 -0.030222   
1        8.502958   42.563595  10.193396   3.329270  24.033822  9.500000   
2       10.068224   45.124179  10.000000   6.207528  23.000000  0.134158   
3        5.192135   46.960747  10.285499   7.885413  23.519880  7.000000   
4        6.213900   38.967323   9.665789  -5.968614  23.368636  9.100000   
...           ...         ...        ...        ...        ...       ...   
8109     6.416863   51.625240   9.681436  10.373504  23.510786  5.900000   
8110     2.647704   46.201928  15.000000   3.026479  25.000000  0.478166   
8111     2.107809   49.482091   9.512278   3.506271  22.796961  8.100000   
8112    27.000000  211.000000  11.000000  27.000000  29.000000  1.200000   
8113     8.918388   61.398460  12.000000  -3.897462  22.000000  0.803937   

                                                                        \
itemid     50893       50902     50912       50931     50960     50970   
0       8.712124  103.000000  1.300000  116.000000  1.720807  2.909850   
1       8.849326   97.341420  0.555639   71.902029  1.735081  3.079829   
2       7.700000  103.000000  0.700000   78.000000  1.500000  1.500000   
3       8.571769   97.662939  0.546662   73.690120  1.711040  3.021489   
4       8.715162   98.577412  0.567741   73.373483  1.823347  2.865983   
...          ...         ...       ...         ...       ...       ...   
8109    8.270673   97.266194  0.519337   72.999882  1.655599  2.873887   
8110    9.000000  106.000000  0.900000  106.000000  2.300000  4.200000   
8111    8.611315   98.796909  0.377824   73.245093  1.784712  2.844369   
8112    8.400000   97.000000  1.000000   90.000000  2.000000  3.300000   
8113    8.100000  106.000000  0.700000  115.000000  1.900000  2.400000   

                                                                         \
itemid     50971       50983      51006     51146      51221      51222   
0       3.500000  140.000000  40.000000  0.390133  39.300000  12.900000   
1       3.602654  134.225802   7.276216  0.199821  39.464173  13.487843   
2       3.600000  135.000000  17.000000  0.293749  34.600000  11.800000   
3       3.497371  132.807198   8.349347  0.330113  39.397553  12.860433   
4       3.523774  135.817396   7.949785  0.413042  36.929292  13.047198   
...          ...         ...        ...       ...        ...        ...   
8109    3.625218  134.687267   8.662937  0.135768  38.812549  13.079045   
8110    4.300000  142.000000  14.000000 -0.077239  43.800000  14.700000   
8111    3.792310  135.306818   8.838752  0.337455  39.960258  13.267441   
8112    3.900000  135.000000  16.000000  0.200000  44.400000  14.000000   
8113    4.400000  136.000000  10.000000  0.003357  29.000000  10.100000   

                                                                          \
itemid     51237      51244      51248      51249      51250       51265   
0       2.900000  22.769974  31.500000  32.900000  96.000000   95.000000   
1       0.906960  24.327817  26.762869  31.574106  80.745531  199.690059   
2       1.300000  19.710321  29.400000  34.100000  86.000000  215.000000   
3       0.943063  21.112103  26.958905  31.332047  84.588624  136.373125   
4       0.909149  26.195502  27.077019  31.592700  85.221055  148.943883   
...          ...        ...        ...        ...        ...         ...   
8109    0.940601  27.073886  27.139609  32.234841  82.105933  172.711736   
8110    0.917266  26.532383  28.900000  33.600000  86.000000  181.000000   
8111    0.912956  23.237294  26.750093  32.055179  83.256674  179.775561   
8112    1.100000  24.200000  26.200000  31.200000  83.000000  321.000000   
8113    0.933333  23.028818  30.100000  33.200000  90.000000  320.000000   

                                                                          \
itemid      51274    

In [1336]:
lab_events_impute_KNN = lab_events_impute.copy()

In [1337]:
lab_events_impute_KNN

min                                                                 \
itemid 50861  50863 50868 50878 50882 50885 50893  50902 50912  50931 50960   
0        NaN    NaN  11.0   NaN  28.0   NaN   NaN  103.0   1.3  116.0   NaN   
1        NaN    NaN   NaN   NaN   NaN   9.5   NaN    NaN   NaN    NaN   NaN   
2        NaN    NaN  10.0   NaN  23.0   NaN   7.7  103.0   0.7   78.0   1.5   
3        NaN    NaN   NaN   NaN   NaN   7.0   NaN    NaN   NaN    NaN   NaN   
4        NaN    NaN   NaN   NaN   NaN   9.1   NaN    NaN   NaN    NaN   NaN   
...      ...    ...   ...   ...   ...   ...   ...    ...   ...    ...   ...   
8109     NaN    NaN   NaN   NaN   NaN   5.9   NaN    NaN   NaN    NaN   NaN   
8110     NaN    NaN  15.0   NaN  25.0   NaN   9.0  106.0   0.9  106.0   2.3   
8111     NaN    NaN   NaN   NaN   NaN   8.1   NaN    NaN   NaN    NaN   NaN   
8112    27.0  211.0  11.0  27.0  29.0   1.2   8.4   97.0   1.0   90.0   2.0   
8113     NaN    NaN  12.0   NaN  22.0   NaN   8.1  106.0   0.7  115.0   1.9   

                                                                           \
itemid 50970 50971  50983 51006 51146 51221 51222 51237 51244 51248 51249   
0        NaN   3.5  140.0  40.0   NaN  39.3  12.9   2.9   NaN  31.5  32.9   
1        NaN   NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2        1.5   3.6  135.0  17.0   NaN  34.6  11.8   1.3   NaN  29.4  34.1   
3        NaN   NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4        NaN   NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
...      ...   ...    ...   ...   ...   ...   ...   ...   ...   ...   ...   
8109     NaN   NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
8110     4.2   4.3  142.0  14.0   NaN  43.8  14.7   NaN   NaN  28.9  33.6   
8111     NaN   NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
8112     3.3   3.9  135.0  16.0   0.2  44.4  14.0   1.1  24.2  26.2  31.2   
8113     2.4   4.4  136.0  10.0   NaN  29.0  10.1   NaN   NaN  30.1  33.2   

                                                                           \
itemid 51250  51265 51274 51275 51277 51279 51301 51464 51466 51478 51484   
0       96.0   95.0  29.4  35.5  14.6  4.10   4.9   NaN   NaN   NaN   NaN   
1        NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2       86.0  215.0  14.8  32.2  13.7  4.01  11.3   NaN   NaN   NaN   NaN   
3        NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4        NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
...      ...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
8109     NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
8110    86.0  181.0   NaN   NaN  15.0  5.08   6.6   NaN   NaN   NaN   NaN   
8111     NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
8112    83.0  321.0  12.2  30.7  14.2  5.32   6.1   NaN   NaN   NaN   NaN   
8113    90.0  320.0   NaN   NaN  12.9  3.33   8.6   NaN   NaN   NaN   NaN   

                                                   max                     \
itemid 51486 51487 51491 51492 51498 51506 51514 50861  50863 50868 50878   
0        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  13.0   NaN   
1        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN   NaN   NaN   
2        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  14.0   NaN   
3        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN   NaN   NaN   
4        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN   NaN   NaN   
...      ...   ...   ...   ...   ...   ...   ...   ...    ...   ...   ...   
8109     NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN   NaN   NaN   
8110     NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  15.0   NaN   
8111     NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN   NaN   NaN   
8112     NaN   NaN   NaN   NaN   NaN   NaN   NaN  27.0  211.0  14.0  27.0   
8113     NaN   NaN   NaN   NaN   NaN   NaN   NaN   Na

In [1338]:
missing_vals = lab_events_impute_KNN.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()

             itemid
above_max    51221     1995
below_min    51221     1995
abn_percent  51221     1995
max          51221     1996
mean         51221     1996
                       ... 
abn_percent  51466     5872
above_max    51514     5872
min          51498     5900
mean         51498     5900
max          51498     5900
Length: 240, dtype: int64

In [1339]:
%%time

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=10)
lab_events_impute_KNN_imputed = imputer.fit_transform(lab_events_impute_KNN)

# 3min 8s

CPU times: user 2min 41s, sys: 30.5 s, total: 3min 12s
Wall time: 3min 8s


In [1340]:
lab_events_impute_KNN_imputed = pd.DataFrame(lab_events_impute_KNN_imputed)
lab_events_impute_KNN_imputed.columns = lab_events_impute_KNN.columns
lab_events_impute_KNN_imputed

min                                                                  \
itemid 50861  50863 50868  50878 50882 50885 50893  50902 50912  50931 50960   
0       25.8   77.0  11.0   49.4  28.0  2.76  7.80  103.0  1.30  116.0  1.80   
1       29.2   77.3  13.7   42.5  22.3  9.50  8.54   95.9  2.36   86.4  1.75   
2       18.4  114.1  10.0   25.7  23.0  1.60  7.70  103.0  0.70   78.0  1.50   
3       61.5   92.3  10.6   33.2  19.7  7.00  8.42  100.4  0.76   82.6  1.82   
4       39.3   86.8   9.9   28.2  23.5  9.10  8.12  102.0  1.18   86.4  1.83   
...      ...    ...   ...    ...   ...   ...   ...    ...   ...    ...   ...   
8109    80.2  112.0  12.3  191.1  23.1  5.90  8.36  100.4  1.07  114.2  1.99   
8110    36.4   71.5  15.0   37.2  25.0  2.21  9.00  106.0  0.90  106.0  2.30   
8111    45.2  104.1  12.3   32.4  25.3  8.10  8.25   99.8  1.65   86.6  1.90   
8112    27.0  211.0  11.0   27.0  29.0  1.20  8.40   97.0  1.00   90.0  2.00   
8113    48.9  126.7  12.0   62.1  22.0  2.95  8.10  106.0  0.70  115.0  1.90   

                                                                         \
itemid 50970 50971  50983 51006 51146  51221  51222 51237  51244  51248   
0       2.37  3.50  140.0  40.0  0.16  39.30  12.90  2.90  17.32  31.50   
1       2.91  3.70  132.5  24.9  0.23  33.93  11.44  1.12  21.68  30.09   
2       1.50  3.60  135.0  17.0  0.35  34.60  11.80  1.30  13.36  29.40   
3       3.29  4.01  134.9  13.3  0.48  33.14  11.36  1.12  24.68  31.65   
4       2.85  3.83  137.5  16.2  0.15  32.21  11.25  1.19  17.99  29.90   
...      ...   ...    ...   ...   ...    ...    ...   ...    ...    ...   
8109    3.28  3.81  135.9  20.5  0.13  32.29  10.85  1.20  23.10  29.25   
8110    4.20  4.30  142.0  14.0  0.25  43.80  14.70  1.23  18.87  28.90   
8111    3.14  3.47  137.8  14.8  0.35  36.42  12.07  1.20  18.85  29.64   
8112    3.30  3.90  135.0  16.0  0.20  44.40  14.00  1.10  24.20  26.20   
8113    2.40  4.40  136.0  10.0  0.36  29.00  10.10  1.32  16.94  30.10   

                                                                           \
itemid  51249 51250  51265  51274  51275  51277  51279  51301 51464 51466   
0       32.90  96.0   95.0  29.40  35.50  14.60  4.100   4.90   0.2   1.1   
1       33.19  89.6  170.6  12.45  28.54  14.43  3.760   5.96   0.2   1.6   
2       34.10  86.0  215.0  14.80  32.20  13.70  4.010  11.30   0.1   0.9   
3       32.62  93.5  179.4  12.39  29.53  14.40  3.466  10.01   0.2   1.3   
4       33.16  87.1  182.9  13.21  30.23  14.72  3.665   6.80   0.0   0.3   
...       ...   ...    ...    ...    ...    ...    ...    ...   ...   ...   
8109    33.10  87.0  193.3  14.70  26.91  13.29  3.699   9.11   0.1   0.7   
8110    33.60  86.0  181.0  13.73  30.27  15.00  5.080   6.60   0.0   0.5   
8111    32.79  89.3  298.8  13.40  29.01  14.35  4.059   9.50   0.1   1.3   
8112    31.20  83.0  321.0  12.20  30.70  14.20  5.320   6.10   0.0   1.2   
8113    33.20  90.0  320.0  14.54  29.68  12.90  3.330   8.60   0.2   0.6   

                                                                   max         \
itemid  51478 51484 51486 51487 51491 51492   51498 51506 51514  50861  50863   
0       110.0   0.0   1.2   0.1  5.60  80.0  1.0167   0.7  0.30   35.7   98.5   
1        10.0   2.0   0.4   0.1  6.40  72.0  1.0171   0.2  1.20   37.0   86.3   
2       100.0   2.0   0.8   0.1  5.85  22.0  1.0194   0.5  0.20   26.5  136.2   
3        10.0  16.0   1.5   0.1  5.65  51.0  1.0112   0.6  0.30  312.9  124.7   
4         0.0  14.5   0.3   0.0  5.80   6.0  1.0134   0.1  0.00  247.6  114.3   
...       ...   ...   ...   ...   ...   ...     ...   ...   ...    ...    ...   
8109      0.0   0.0   0.1   0.0  6.10  18.0  1.0183   0.1  0.20  305.6  119.6   
8110      0.0  14.5   0.6   0.0  6.00  43.5  1.0201   0.3  0.40   47.4   82.2   
8111     15.0   1.0   0.9   0.0  6.20  16.0  1.0175   0.3  0.00   49.4  112.6   
8112     15.0   1.0   0.8   0.0  6.10  13.0  1.0161   0.3  0.00   27.0  211.0   
8113      0.0   2.

## Standardize the Data for use in Models

In [1341]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()


In [1342]:
labs_scaled = lab_events_sampled_pivot
labs_scaled_KNN = lab_events_impute_KNN_imputed


In [1343]:
labs_scaled

min                                                         \
itemid      50861       50863      50868      50878      50882     50885   
0        2.997088   51.875587  11.000000   7.192662  28.000000 -0.030222   
1        8.502958   42.563595  10.193396   3.329270  24.033822  9.500000   
2       10.068224   45.124179  10.000000   6.207528  23.000000  0.134158   
3        5.192135   46.960747  10.285499   7.885413  23.519880  7.000000   
4        6.213900   38.967323   9.665789  -5.968614  23.368636  9.100000   
...           ...         ...        ...        ...        ...       ...   
8109     6.416863   51.625240   9.681436  10.373504  23.510786  5.900000   
8110     2.647704   46.201928  15.000000   3.026479  25.000000  0.478166   
8111     2.107809   49.482091   9.512278   3.506271  22.796961  8.100000   
8112    27.000000  211.000000  11.000000  27.000000  29.000000  1.200000   
8113     8.918388   61.398460  12.000000  -3.897462  22.000000  0.803937   

                                                                        \
itemid     50893       50902     50912       50931     50960     50970   
0       8.712124  103.000000  1.300000  116.000000  1.720807  2.909850   
1       8.849326   97.341420  0.555639   71.902029  1.735081  3.079829   
2       7.700000  103.000000  0.700000   78.000000  1.500000  1.500000   
3       8.571769   97.662939  0.546662   73.690120  1.711040  3.021489   
4       8.715162   98.577412  0.567741   73.373483  1.823347  2.865983   
...          ...         ...       ...         ...       ...       ...   
8109    8.270673   97.266194  0.519337   72.999882  1.655599  2.873887   
8110    9.000000  106.000000  0.900000  106.000000  2.300000  4.200000   
8111    8.611315   98.796909  0.377824   73.245093  1.784712  2.844369   
8112    8.400000   97.000000  1.000000   90.000000  2.000000  3.300000   
8113    8.100000  106.000000  0.700000  115.000000  1.900000  2.400000   

                                                                         \
itemid     50971       50983      51006     51146      51221      51222   
0       3.500000  140.000000  40.000000  0.390133  39.300000  12.900000   
1       3.602654  134.225802   7.276216  0.199821  39.464173  13.487843   
2       3.600000  135.000000  17.000000  0.293749  34.600000  11.800000   
3       3.497371  132.807198   8.349347  0.330113  39.397553  12.860433   
4       3.523774  135.817396   7.949785  0.413042  36.929292  13.047198   
...          ...         ...        ...       ...        ...        ...   
8109    3.625218  134.687267   8.662937  0.135768  38.812549  13.079045   
8110    4.300000  142.000000  14.000000 -0.077239  43.800000  14.700000   
8111    3.792310  135.306818   8.838752  0.337455  39.960258  13.267441   
8112    3.900000  135.000000  16.000000  0.200000  44.400000  14.000000   
8113    4.400000  136.000000  10.000000  0.003357  29.000000  10.100000   

                                                                          \
itemid     51237      51244      51248      51249      51250       51265   
0       2.900000  22.769974  31.500000  32.900000  96.000000   95.000000   
1       0.906960  24.327817  26.762869  31.574106  80.745531  199.690059   
2       1.300000  19.710321  29.400000  34.100000  86.000000  215.000000   
3       0.943063  21.112103  26.958905  31.332047  84.588624  136.373125   
4       0.909149  26.195502  27.077019  31.592700  85.221055  148.943883   
...          ...        ...        ...        ...        ...         ...   
8109    0.940601  27.073886  27.139609  32.234841  82.105933  172.711736   
8110    0.917266  26.532383  28.900000  33.600000  86.000000  181.000000   
8111    0.912956  23.237294  26.750093  32.055179  83.256674  179.775561   
8112    1.100000  24.200000  26.200000  31.200000  83.000000  321.000000   
8113    0.933333  23.028818  30.100000  33.200000  90.000000  320.000000   

                                                                          \
itemid      51274    

In [1344]:
scaler.fit(labs_scaled.loc[:, ('min', 50861) : ('above_max', 51514)])
labs_scaled.loc[:, ('min', 50861) : ('above_max', 51514)] = scaler.transform(labs_scaled.loc[:, ('min', 50861) : ('above_max', 51514)])

scaler.fit(labs_scaled_KNN.loc[:, ('min', 50861) : ('above_max', 51514)])
labs_scaled_KNN.loc[:, ('min', 50861) : ('above_max', 51514)] = scaler.transform(labs_scaled_KNN.loc[:, ('min', 50861) : ('above_max', 51514)])


In [1345]:
pd.DataFrame(labs_scaled_KNN)

min                                                              \
itemid     50861     50863     50868     50878     50882     50885     50893   
0      -0.266859 -0.397199 -0.347562  0.087319  1.469562 -0.065554 -0.868565   
1      -0.222071 -0.391690  0.774897 -0.031307 -0.154531  2.070029  0.295981   
2      -0.364340  0.284059 -0.763287 -0.320137  0.044919 -0.433103 -1.025936   
3       0.203419 -0.116249 -0.513852 -0.191195 -0.895345  1.277899  0.107136   
4      -0.089023 -0.217244 -0.804860 -0.277156  0.187384  1.943288 -0.364977   
...          ...       ...       ...       ...       ...       ...       ...   
8109    0.449755  0.245498  0.192881  2.523458  0.073412  0.929361  0.012713   
8110   -0.127225 -0.498194  1.315340 -0.122426  0.614776 -0.239823  1.019889   
8111   -0.011302  0.100432  0.192881 -0.204949  0.700255  1.626436 -0.160395   
8112   -0.251052  2.063411 -0.347562 -0.297787  1.754491 -0.559843  0.075662   
8113    0.037439  0.515430  0.068164  0.305661 -0.240009 -0.005352 -0.396451   

                                                                              \
itemid     50902     50912     50931     50960     50970     50971     50983   
0       0.581629  0.508489  0.684559 -0.126524 -0.723958 -0.636273  0.937203   
1      -1.065383  2.053808 -0.476959 -0.321136 -0.002565 -0.165694 -1.046233   
2       0.581629 -0.366219 -0.806578 -1.294195 -1.886203 -0.400984 -0.385087   
3      -0.021502 -0.278748 -0.626072 -0.048679  0.505082  0.563703 -0.411533   
4       0.349656  0.333548 -0.476959 -0.009757 -0.082720  0.140182  0.276058   
...          ...       ...       ...       ...       ...       ...       ...   
8109   -0.021502  0.173185  0.613926  0.613001  0.491723  0.093124 -0.147075   
8110    1.277549 -0.074650  0.292154  1.819594  1.720763  1.246042  1.466119   
8111   -0.160686  1.018736 -0.469111  0.262700  0.304695 -0.706860  0.355395   
8112   -0.810212  0.071135 -0.335693  0.651923  0.518441  0.304884 -0.385087   
8113    1.277549 -0.366219  0.645318  0.262700 -0.683881  1.481331 -0.120629   

                                                                              \
itemid     51006     51146     51221     51222     51237     51244     51248   
0       2.761055 -0.600666  0.784658  0.634403  7.453598 -0.198872  0.475357   
1       1.081170 -0.319866  0.040925  0.057151 -0.283621  0.256838 -0.055582   
2       0.202290  0.161505  0.133719  0.199487  0.498794 -0.612773 -0.315403   
3      -0.209337  0.682990 -0.068488  0.025521 -0.283621  0.570399  0.531840   
4       0.113289 -0.640780 -0.197291 -0.017971  0.020652 -0.128843 -0.127127   
...          ...       ...       ...       ...       ...       ...       ...   
8109    0.591667 -0.721009 -0.186211 -0.176122  0.064119  0.405257 -0.371886   
8110   -0.131462 -0.239637  1.407898  1.346083  0.194522 -0.036865 -0.503679   
8111   -0.042461  0.161505  0.385785  0.306239  0.064119 -0.038955 -0.225031   
8112    0.091039 -0.440209  1.490997  1.069318 -0.370556  0.520229 -1.520371   
8113   -0.576465  0.201619 -0.641868 -0.472655  0.585729 -0.238589 -0.051817   

                                                                              \
itemid     51249     51250     51265     51274     51275     51277     51279   
0       0.073734  0.750795 -1.429922  5.712699  0.823431  0.211976  0.568440   
1       0.280128 -0.167762 -0.471561 -0.266796 -0.205069  0.108194  0.103351   
2       0.927776 -0.684451  0.091286  0.562219  0.335780 -0.337460  0.445328   
3      -0.125543  0.391983 -0.360006 -0.287963 -0.058773  0.089879 -0.298813   
4       0.258777 -0.526574 -0.315637  0.001311  0.044668  0.285234 -0.026600   
...          ...       ...       ...       ...       ...       ...       ...   
8109    0.216074 -0.540926 -0.183799  0.526942 -0.445939 -0.587759  0.019909   
8110    0.571925 -0.684451 -0.339723  0.184753  0.050579  0.456170  1.908989   
8111   -0.004553 -0.210820  1.153596  0.068338 -0.135615  0.059355  0.512356   
8112   -1.

## Check for Multicollinearity

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [ ]:
%%time

labs_scaled_features = labs_scaled.loc[:, ('min', 50868) : ('above_max', 51484)]
add_constant(labs_scaled_features)

high_vif = pd.Series([variance_inflation_factor(labs_scaled_features.values, i) 
               for i in range(labs_scaled_features.shape[1])], 
              index=labs_scaled_features.columns).sort_values(ascending = False)

high_vif.head(20)


In [ ]:
labs_scaled_features_limited = labs_scaled_features.copy()
labs_scaled_features_limited.drop([('above_max', 51484),
                                  ('above_max', 51478),
                                  ('max', 51237),
                                  ('mean', 51248)], axis = 1, inplace = True)


In [ ]:
%%time

high_vif = pd.Series([variance_inflation_factor(labs_scaled_features_limited.values, i) 
               for i in range(labs_scaled_features_limited.shape[1])], 
              index=labs_scaled_features_limited.columns).sort_values(ascending = False)

high_vif.head(20)


## Save Lab Results for Models

In [1346]:
AKF_final_test = labs_scaled.merge(admissions_sample, how='left', left_on='hadm_id', right_on='hadm_id')
AKF_KNN_final_test = labs_scaled_KNN.merge(admissions_sample, how='left', left_on='hadm_id', right_on='hadm_id')


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [1347]:
# admissions_sample.to_pickle("admissions_sample.pkl")
# admissions_sample.to_pickle("admissions_sample.pkl")
AKF_final_test.to_csv('AKF_final_test.csv.gz', compression="gzip")
AKF_KNN_final_test.to_csv('AKF_KNN_final_test.csv.gz', compression="gzip")
